# zhsegment: default program

In [4]:
from default import *

## Run the default solution on dev

In [5]:
Pw = Pdist(data=datafile("data/count_1w.txt"), missingfn=avoid_long_words)
segmenter = UnigramSegmenter(Pw) # note that the default solution for this homework ignores the unigram counts
output_full = []
with open("data/input/dev.txt") as f:
    for line in f:
        output = " ".join(segmenter.segment(line.strip()))
        output_full.append(output)
print("\n".join(output_full[:3])) # print out the first three lines of output as a sanity check

中 美 在 沪 签订 高 科技 合作 协议
新华社 上海 八月 三十一日 电 （ 记者 白 国 良 、 夏儒阁 ）
“ 中 美 合作 高 科技 项目 签字 仪式 ” 今天 在 上海 举行 。


## Evaluate the default output

In [7]:
from zhsegment_check import fscore
with open('data/reference/dev.out', 'r') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
    tally = fscore(ref_data, output_full)
    print("score: {:.4f}".format(tally), file=sys.stderr)


score: 0.9253


## Documentation

#### UnigramSegmenter Class

Class object that constructs a unigram language model to segment text.


#### BigramSegmenter Class

Class object that constructs a bigram language model to segment text.

#### avoid_long_words Function

Unigram smoothing function that estimates the probability of an unknown word. This function peanlizes words with a greater length. (Modified code from P. Norvig's - "Beautiful Data")

## Analysis

#### Unigram Segmenter

The first major hurdle we ran into was when building the unigram segmenter class with no smoothing function. Our initial solution resulted in a dev score of 0.03 and output segments consistently being 5-6 characters long. Due to the segmentation output, we thought that this was a result of a bug in the unigram segmenter class code, but this was not the case.

After a lot of debugging and experimentation, we found that the code base was correct, but we needed a smoothing function to achieve a competitive score. We found that using a modified version of P. Norvig's smoothing function from the previous assignment achieved much better results. For the Chinese language, we found that penalizing large words more heavily resulted in better results when compared with English language segmentation. This could be due to a higher frequency of short words in the Chinese language when compared with the English language. The best dev score we were able to achieve using the unigram segmenter class was ~0.93.

Correct Answer
```
中 美 在 沪 签订 高 科技 合作 协议
新华社 上海 八月 三十一日 电 （ 记者 白国良 、 夏儒阁 ）
“ 中 美 合作 高 科技 项目 签字 仪式 ” 今天 在 上海 举行 。
```

Without Smoothing Function
```
中 美在沪签订高 科技合作协议
新华社上海八 月三十一日电 （记者白国 良、夏儒阁）
“中美合作 高科技项目签 字仪式”今天 在上海举行
```

With Smoothing Function
```
中 美 在 沪 签订 高 科技 合作 协议
新华社 上海 八月 三十一日 电 （ 记者 白 国 良 、 夏儒阁 ）
“ 中 美 合作 高 科技 项目 签字 仪式 ” 今天 在 上海 举行 。
```

#### Bigram Segmenter

The second iteration of the codebase was to implement a bigram model. The only difficulty we had here was choosing the correct smoothing function. We first implemented additive smoothing, but it yielded poor results (0.79). We then tried back-off smoothing which worked much better. We suspect that this is because the unigram model we build before was highly optimized. Our final iteration of the bigram model uses a modified backoff smoothing function that can be found on the slides [here](http://www.cs.cornell.edu/courses/cs4740/2014sp/lectures/smoothing+backoff.pdf). This function adds an alpha parameter that scales the unigram probability when the complete bigram is not found. 

Similar to the unigram class above, we can compare the correct segmentation output with the segmentation that was generated by the bigram model. The best dev score we achieved with this model was ~0.93. Interestingly this model is still slightly outperformed by the unigram model, but believe that this model will be more generalizable to the test set. We can see the output of the bigram model when compared with the true output below.

Correct Output
```
美国 商务部长 布朗 访问 上海
新华社 上海 八月 三十一日 电 （ 记者 周解蓉 ）
```

Bigram Output
```
美国 商务 部长 布朗 访问 上海
新华社 上海 八月 三十一日 电 （ 记者 周解蓉 ）
```